In [18]:
!git clone https://github.com/UKPLab/sentence-transformers.git

Cloning into 'sentence-transformers'...
remote: Enumerating objects: 4679, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 4679 (delta 114), reused 145 (delta 58), pack-reused 4455
Receiving objects: 100% (4679/4679), 15.97 MiB | 17.34 MiB/s, done.
Resolving deltas: 100% (3209/3209), done.


In [1]:
import torch
import sys
import logging

sys.path.append("../KoBERT")
from kobert.pytorch_kobert import get_pytorch_kobert_model
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
import transformers

# Model, Tokenizer load

In [2]:
logging.info("load model and tokenizer")
# model, vocab load
model, vocab  = get_pytorch_kobert_model()

# tokenizer load
tok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)

using cached model
using cached model
using cached model


# 학습 데이터 load

In [3]:
from sentence_transformers import InputExample, losses
from torch.utils.data import DataLoader
import logging
import pandas as pd

In [5]:
logging.info("Read AllNLI train dataset")

label2int = {"contradiction": 0, "entailment": 1, "neutral": 2}
train_samples = []

train_df = pd.read_csv('../data/KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv', sep='\t', encoding="utf-8")

for s1, s2, labels in zip(train_df['sentence1'], train_df['sentence2'], train_df['gold_label']):
    label = label2int[labels.strip()]
    train_samples.append(InputExample(texts=[s1, s2], label=label))

In [6]:
vocab.token_to_idx['[SEP]']

3

In [7]:
# NLI Dataset 선언
class SentencesDataset(torch.utils.data.Dataset):
    def __init__(self, examples, tokenizer, vocab, max_seq_length):
        '''
        examples : List[InputExample]
        tokenizer : SentencepieceTokenizer
        vocab : vocab module
        max_seq_length : max sequence length.
        '''
        self.examples = examples
        self.label_type = torch.long if isinstance(self.examples[0].label, int) else torch.float
        self.tokenizer = tokenizer
        self.vocab = vocab
        self.max_seq_length = max_seq_length
    
    def __getitem__(self, idx):
        item = dict()
        # tokenizing
        s1, s2 = [self.tokenizer(text) for text in self.examples[idx].texts]
        
        # slicing(consider special token)
        s1 = s1[:self.max_seq_length-2] if len(s1) > self.max_seq_length-2 else s1
        s2 = s2[:self.max_seq_length-2] if len(s2) > self.max_seq_length-2 else s2
        
        # add special token
        s1 = ['[CLS]'] + s1 + ['[SEP]']
        s2 = ['[CLS]'] + s2 + ['[SEP]']
        
        # token to index
        s1 = [vocab.token_to_idx[x] for x in s1]
        s2 = [vocab.token_to_idx[x] for x in s2]
        
        # add padding
        s1_ = s1 + [0]*(self.max_seq_length - len(s1)) if len(s1) < self.max_seq_length else s1[:self.max_seq_length]
        s2_ = s2 + [0]*(self.max_seq_length - len(s2)) if len(s2) < self.max_seq_length else s2[:self.max_seq_length]
        
        item['input_ids1'] = torch.tensor(s1_)
        item['input_ids2'] = torch.tensor(s2_)
        
        # attention mask 
        attention_mask1 = [1] * len(s1) + [0] * (self.max_seq_length - len(s1)) if len(s1) < self.max_seq_length else [1] * self.max_seq_length
        attention_mask2 = [1] * len(s2) + [0] * (self.max_seq_length - len(s2)) if len(s2) < self.max_seq_length else [1] * self.max_seq_length
        
        item['attention_mask1'] = torch.tensor(attention_mask1)
        item['attention_mask2'] = torch.tensor(attention_mask2)
        
        # label
        item['label'] = torch.tensor(self.examples[idx].label, dtype=self.label_type) 
        
        item['token_type_ids1'] = torch.tensor(self.max_seq_length * [0])
        item['token_type_ids2'] = torch.tensor(self.max_seq_length * [0])
        return item
            
    def __len__(self):
        return len(self.examples)

In [8]:
train_dataset = SentencesDataset(train_samples, sp, vocab, 30)
train_dataset[0]

{'input_ids1': tensor([   2, 1962, 4707, 2589,  993, 7178, 5663, 2545, 3554, 1868, 6964, 6115,
         5782,   54,    3,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]),
 'input_ids2': tensor([   2, 4955, 2589,  970, 7088, 3567, 1962, 5184, 6723, 3862,   54,    3,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]),
 'attention_mask1': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]),
 'attention_mask2': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]),
 'label': tensor(2),
 'token_type_ids1': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]),
 'token_type_ids2': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0])}

In [9]:
train_batch_size = 2
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

In [10]:
next(iter(train_dataloader))

{'input_ids1': tensor([[   2, 4501, 6083, 6538,  517, 6601, 7478, 6983, 4799, 6016, 6538, 2207,
          6273, 7329, 3843, 4073, 3313, 7096, 3454, 7078,  745, 4425, 3006, 6093,
          7748, 4636, 6116, 1802, 4955,    3],
         [   2, 2586, 4303, 5330, 5760, 2369, 3552, 6896, 1773, 2038,  976, 7096,
          2718, 3862,   54,    3,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0]]),
 'input_ids2': tensor([[   2, 4073, 3313, 7086, 1495, 2207, 6273, 7318, 6896, 4799, 6016,  517,
          6601, 7478, 6116, 3838, 3862,    3,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0],
         [   2, 1487, 7096, 1958, 7788, 3873,    3,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0]]),
 'attention_mask1': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1

In [11]:
# SoftmaxLoss(NLI datasets)
class SoftmaxLoss(torch.nn.Module):
    """
    Reference
    https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/SoftmaxLoss.py
    
    This loss was used in our SBERT publication (https://arxiv.org/abs/1908.10084) to train the SentenceTransformer
    model on NLI data. It adds a softmax classifier on top of the output of two transformer networks.

    :param model: pretrained model
    :param sentence_embedding_dimension: Dimension of your sentence embeddings
    :param num_labels: Number of different labels
    :param concatenation_sent_rep: Concatenate vectors u,v for the softmax classifier?
    :param concatenation_sent_difference: Add abs(u-v) for the softmax classifier?
    :param concatenation_sent_multiplication: Add u*v for the softmax classifier?
    
    """
    def __init__(self, model, sentence_embedding_dimension, num_labels, concatenation_sent_rep, concatenation_sent_difference, concatenation_sent_multiplication):
        super(SoftmaxLoss, self).__init__()
        self.model = model
        self.num_labels = num_labels
        self.concatenation_sent_rep = concatenation_sent_rep
        self.concatenation_sent_difference = concatenation_sent_difference
        self.concatenation_sent_multiplication = concatenation_sent_multiplication
        
        num_vectors_concatenated = 0
        if concatenation_sent_rep:
            num_vectors_concatenated += 2
        if concatenation_sent_difference:
            num_vectors_concatenated += 1
        if concatenation_sent_multiplication:
            num_vectors_concatenated += 1
        logging.info("Softmax loss: #Vectors concatenated: {}".format(num_vectors_concatenated))
        self.classifier = torch.nn.Linear(num_vectors_concatenated * sentence_embedding_dimension, num_labels)
    
    def mean_pooling(self, token_embeddings, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask
        
    def forward(self, x):
        # input_ids1, attention_mask1, token_type_ids1, input_ids2, attention_mask2, token_type_ids2, labels
        output1 = self.model(input_ids=x['input_ids1'], attention_mask=x['attention_mask1'], token_type_ids=x['token_type_ids1'])
        output2 = self.model(input_ids=x['input_ids2'], attention_mask=x['attention_mask2'], token_type_ids=x['token_type_ids2'])
        
        rep_a = self.mean_pooling(output1[0], x['attention_mask1'])
        rep_b = self.mean_pooling(output2[0], x['attention_mask2'])
        
        vectors_concat = []
        if self.concatenation_sent_rep:
            vectors_concat.append(rep_a)
            vectors_concat.append(rep_b)

        if self.concatenation_sent_difference:
            vectors_concat.append(torch.abs(rep_a - rep_b))

        if self.concatenation_sent_multiplication:
            vectors_concat.append(rep_a * rep_b)

        features = torch.cat(vectors_concat, 1)
        
        output = self.classifier(features)
        loss_fct = torch.nn.CrossEntropyLoss()
        
        if labels is not None:
            loss = loss_fct(output, x['label'].view(-1))
            return loss
        else:
            return reps, output

# NLI 학습 및 검증

In [13]:
# NLI dataset load
logging.info("Read AllNLI train dataset")

label2int = {"contradiction": 0, "entailment": 1, "neutral": 2}
train_samples = []

train_df = pd.read_csv('../data/KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv', sep='\t', encoding="utf-8")

for s1, s2, labels in zip(train_df['sentence1'], train_df['sentence2'], train_df['gold_label']):
    label = label2int[labels.strip()]
    train_samples.append(InputExample(texts=[s1, s2], label=label))


batch_size = 16
max_sequence_length = 64

# train dataset Load
train_dataset = SentencesDataset(train_samples, sp, vocab, max_sequence_length)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

In [14]:
# validation dataset load
logging.info("Read STSbenchmark dev dataset")
dev_samples = []

with open('../data/KorNLUDatasets/KorSTS/tune_dev.tsv', 'rt', encoding='utf-8') as fIn:
    lines = fIn.readlines()
    for line in lines:
        s1, s2, score = line.split('\t')
        score = score.strip()
        score = float(score) / 5.0
        dev_samples.append(InputExample(texts= [s1,s2], label=score))

In [15]:
dev_dataset = SentencesDataset(dev_samples, sp, vocab, max_sequence_length)
dev_dataloader = DataLoader(dev_dataset, shuffle=True, batch_size=train_batch_size)

# 학습

In [16]:
batch_size = 16
num_epochs = 1
weight_decay = 0.01

num_training_steps = int(len(train_dataloader) * num_epochs)

import math
# model 선언
SBERT_model = SoftmaxLoss(model, 768, 3, True, True, False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

param_optimizer = list(SBERT_model.named_parameters())
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


# optimizer 선언
from transformers import AdamW

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=1e-6, correct_bias=False)
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, warmup_steps, num_training_steps=num_training_steps)


# NLI dataset 학습
from tqdm import tqdm
from tqdm.autonotebook import trange

# 기록을 위해 tensorboard 사용
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(f'runs/{num_epochs}_{batch_size}')

global_steps = 0
running_loss = 0
for epoch in trange(num_epochs, desc='Epoch', disable=not True):
    training_steps = 0
    
    for batch in tqdm(train_dataloader, desc='Iteration', disable=False):
        SBERT_model.zero_grad()
        SBERT_model.train()
        SBERT_model.to(device)
        
        batch = {key: batch[key].to(device) for key in batch.keys()}
        
        loss_value = SBERT_model(batch)
        loss_value.backward()
        torch.nn.utils.clip_grad_norm(SBERT_model.parameters(), 1)
        optimizer.step()
        scheduler.step()
        
        running_loss += loss_value.item()
        
        if training_steps % 1000 == 999:
            print(f'training_loss : {running_loss / 1000:.4f}, epochs: {epoch}, training_steps: {training_steps}')
            writer.add_scalar('training_loss', running_loss / 1000, training_steps + epoch * len(train_dataloader))
            torch.save(SBERT_model, f'./trained_model/{epoch}_{training_steps}_{running_loss/1000:.4f}.pt')
            running_loss = 0
        
        training_steps += 1
    global_steps += 1

Iteration:   0%|          | 0/275076 [00:00<?, ?it/s]<ipython-input-16-b4f9f52685ca>:51: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(SBERT_model.parameters(), 1)
Iteration:   0%|          | 998/275076 [01:30<6:40:36, 11.40it/s]

training_loss : 1.0999, epochs: 0, training_steps: 999


Iteration:   1%|          | 1999/275076 [03:11<8:23:20,  9.04it/s] 

training_loss : 1.0912, epochs: 0, training_steps: 1999


Iteration:   1%|          | 2999/275076 [04:52<8:24:41,  8.98it/s] 

...

training_loss : 0.8838, epochs: 0, training_steps: 74999


Iteration:  28%|██▊       | 75999/275076 [2:02:05<5:59:42,  9.22it/s] 

training_loss : 0.9102, epochs: 0, training_steps: 75999


Iteration:  28%|██▊       | 76321/275076 [2:02:36<5:19:18, 10.37it/s] 

TypeError: not a string

# Test model

In [ ]:
def mean_pooling(token_embeddings, attention_mask):
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [ ]:
logging.info('STS dataset ')

In [ ]:
next(iter(dev_dataloader))

In [ ]:
len(train_dataloader)